In [1]:
#NOTE:
"uriyas model total emotions: happy, sad, disgust, fear, neutral, anger, surprise"
"use for valence: happy/sad"
"if needed, he'll use for arousal: neutral/suprise"
#THEN, for me:
#map the emotions to (x,y) coordinates of valence and arousal, then multiply it by the probability of the emotion, then take the weighted average and get the reslting x,y coordinates
#valence and arousal are the two dimensions of the circumplex model and the resulting scores should get put in one of the 9(nuetral plus 8 categories) categories

"if needed, he'll use for arousal: neutral/suprise"

# GoEmotions: Testing Input to the Model, Adding Softmax Layer, Conversion Function for getting segment Label, Getting final Label

# 1. Loading Model

In [42]:
import transformers
import tensorflow as tf
import torch

In [3]:
#load the model monologg/bert-base-cased-goemotions-original from huggingface
model = transformers.AutoModelForSequenceClassification.from_pretrained("monologg/bert-base-cased-goemotions-original")
tokenizer = transformers.AutoTokenizer.from_pretrained("monologg/bert-base-cased-goemotions-original")
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [4]:
#test the model on a sentence
sentence = "I am so happy"
tokens = tokenizer.encode(sentence, return_tensors="pt")
tokens = tokens.cuda()
result = model(tokens)

# 2. Playing around/testing model (skip)
this is to figure out what will be needed making softmax, and determining the classes

In [5]:
#determine the max size of the input the model can handle
max_length = model.config.max_position_embeddings
print(max_length)

512


In [6]:
#identify the top 3 emotion with the highest probability
emotion = result.logits
emotion = emotion.cpu().detach().numpy()
emotion = emotion[0]
emotion = emotion.argsort()[-3:][::-1]
emotion = emotion.tolist()
print(emotion)

[17, 0, 13]


In [7]:
#convert model.config.id2label to a list of emotions, where the key is the index of the emotion
emotion_dict = model.config.id2label

In [8]:
#identify the label of top 3 emotions from emotion list
for i in emotion:
    print(emotion_dict[i])


joy
admiration
excitement


In [9]:
#identify the emotion with the highest probability
emotion = result.logits.argmax()
print(emotion)
#which emotion is that?
print(model.config.id2label[emotion.item()])

tensor(17, device='cuda:0')
joy


In [10]:
emotion_dict.values()
emotionGroups=[]

In [11]:
emotionSet = set(emotion_dict.values())

In [12]:
approvedMoods=['neutral']

# 3. Softmax

### Function

In [13]:
def getSoftmax(model, sentence, n,printRawScores=False, printTopN=True):
    tokens = tokenizer.encode(sentence, return_tensors="pt")
    tokens = tokens.cuda()
    result = model(tokens)
    emotion = result.logits
    emotion = emotion.cpu().detach().numpy()
    emotion = emotion[0]
    softmax = tf.nn.softmax(emotion)
    if printRawScores:
        print(softmax)
    
    if printTopN:
        emotion = emotion.argsort()[-n:][::-1]
        emotion = emotion.tolist()
        printTopEmotions(emotion,model, softmax)
    return softmax


def printTopEmotions(emotion, model, softmax):
    
    #identify the label of top n emotions from emotion list
    #softmax is in the order of the values in emotion_dict so we can use emotion[id] to get the softmax value
    id=0
    emotion_dict = model.config.id2label
    for i in emotion:
        print(emotion_dict[i])
        print(softmax[emotion[id]].numpy()*100,"%")
        id+=1
    return

### sentences from songs to test with

In [14]:
unclear2 = "'Cause you see people, people, people, people Don't really know you (They don't really know you) They don't really know you 'Cause you see people, people, people They don't really know you (Mmm) They don't really know—"
unclear = "I've been drinking more alcohol for the past five days Did you check on me? Now, did you look for me? I walked in the room, eyes are red and I don't smoke banga Did you check on me? (Did you check on me?) Now, did you notice me?"

In [15]:
braggingLyrics = "She just hit my phone, she said, 'Tecca, you a winner' (Yeah) Took her home, then I turn her to a sinner, yeah I just got a check, I ate lobster for dinner She like my style, she tryna turn me to her nigga, yeah I just woke up, now they say that I’m a star I just wanna ball with my guys Drive off the Wock', he just caught a DUI He think he know the answers, nigga, like he Bill Nye"

In [16]:
sadMetaphoricLyrics = "Hello darkness, my old friend, I've come to talk with you again, because a vision softly creeping, left its seeds while I was sleeping."

In [17]:
goEasyOnMeSad = "There ain't no gold in this river That I've been washin' my hands in forever I know there is hope in these waters But I can't bring myself to swim When I am drowning in this silence Baby, let me in. Go easy on me, baby I was still a child Didn't get the chance to Feel the world around me I had no time to choose what I chose to do So go easy on me"

### Results

In [18]:
softmaxtestUnclear=getSoftmax(model, unclear, 3)

curiosity
99.90363121032715 %
neutral
0.034640790545381606 %
confusion
0.019302892906125635 %


In [19]:
softmaxtestUnclear2=getSoftmax(model,unclear2, 3)

neutral
99.9980092048645 %
annoyance
0.00058388282013766 %
disappointment
0.0003420888106120401 %


In [20]:
softmaxtestBraggingLyrics=getSoftmax(model,braggingLyrics, 3)

neutral
75.65684914588928 %
joy
16.482388973236084 %
excitement
4.176769405603409 %


In [21]:
softmaxtestsadMetaphoricLyrics=getSoftmax(model,sadMetaphoricLyrics, 3)

neutral
99.99864101409912 %
annoyance
0.00020481845695030643 %
caring
0.00011883536217283108 %


In [22]:
softmaxtestgoEasyOnMeSad=getSoftmax(model,goEasyOnMeSad, 3)

disappointment
63.13481330871582 %
realization
14.627596735954285 %
sadness
14.258602261543274 %


# 3. SoftMax Recap:

1. There seems to be a dominance of neutral while the trailing classes seem to be more relevant. This does make sense considering the fact that not every lyric is emotionally charged.  

- However, this does raise the question, "How do we handle neutral?"

- similarly, "Do we want neutral as a resulting category?

2. Songs that are reliant on metaphores or to create emotion will not be captured via sentiment analysis

- In these cases, we would want the linear model to give results
- perhaps if this is specifially reated neutral we should rely on the linear classificiaton model

# 4. Class Conversion Without Special Neutral Considerations

Here we will:
1. scale emotions to (valence, energy) based on the valence-arousal mood wheel
2. multiply that value by the percent
3. add up x values as well as y values
4. place resulting values into the 8 main mood categories to get a class
5. if close to 0, label it neutral instead
    - If neutral, perhaps use as indicator to rely on 2nd model for classification

NOTE: Some words did not have a perfect point plotted, but there were some synonyms
- approval was mapped with satisfied
- caring was with passionate
- confused with a lessened amount of frustrated (1/2)
- curious was mapped with interested

### Mood setup verification (skip)

In [23]:
emotion_dict = model.config.id2label
for key in range(len(emotion_dict)):
    print(emotion_dict[key])

admiration
amusement
anger
annoyance
approval
caring
confusion
curiosity
desire
disappointment
disapproval
disgust
embarrassment
excitement
fear
gratitude
grief
joy
love
nervousness
optimism
pride
realization
relief
remorse
sadness
surprise
neutral


In [24]:
# make a dictionary mapping all emotions to valence and arousal values
# valence is the x axis, arousal is the y axis of the circumplex model
emotionsAsValenceArousal= { 'admiration':(.6,.4),'amusement':(.6,.2),'anger':(-.8,.6),'annoyance':(-.6,.6),'approval':(.8,.6),'caring':(.6,-.2),'confusion':(-.2,.2),'curiosity':(0,.4),'desire':(.6,.6),'disappointment':(-.6,-.6),'disapproval':(-.8,.65),'disgust':(-.8,.2),'embarrassment':(-.6,.4),'excitement':(.6,.8),'fear':(-.6,.8),'gratitude':(.6,-.6),'grief':(-.6,-.8),'gratitude':(.6,-.6),'grief':(-.6,-.8),'joy':(.8,.2),'love':(.8,.4),'nervousness':(-.4,.6),'optimism':(.6,-.3),'pride':(.6,.1),'realization':(.2,.2),'relief':(.4,-.4),'remorse':(-.6,-.4),'sadness':(-.8,-.2),'surprise':(.2,.6),'neutral':(0,0)}


In [25]:
for i in range(len(emotion_dict)):
    print(emotionsAsValenceArousal[emotion_dict[i]])

(0.6, 0.4)
(0.6, 0.2)
(-0.8, 0.6)
(-0.6, 0.6)
(0.8, 0.6)
(0.6, -0.2)
(-0.2, 0.2)
(0, 0.4)
(0.6, 0.6)
(-0.6, -0.6)
(-0.8, 0.65)
(-0.8, 0.2)
(-0.6, 0.4)
(0.6, 0.8)
(-0.6, 0.8)
(0.6, -0.6)
(-0.6, -0.8)
(0.8, 0.2)
(0.8, 0.4)
(-0.4, 0.6)
(0.6, -0.3)
(0.6, 0.1)
(0.2, 0.2)
(0.4, -0.4)
(-0.6, -0.4)
(-0.8, -0.2)
(0.2, 0.6)
(0, 0)


### Function

In [26]:
emotionsAsValenceArousal= { 'admiration':(.6,.4),'amusement':(.6,.2),'anger':(-.8,.6),'annoyance':(-.6,.6),'approval':(.8,.6),'caring':(.6,-.2),'confusion':(-.2,.2),'curiosity':(0,.4),'desire':(.6,.6),'despair':(-.8,-.6),'disappointment':(-.6,-.6),'disapproval':(-.8,.65),'disgust':(-.8,.2),'embarrassment':(-.6,.4),'envy':(-.6,.4),'excitement':(.6,.8),'fear':(-.6,.8),'gratitude':(.6,-.6),'grief':(-.6,-.8),'gratitude':(.6,-.6),'grief':(-.6,-.8),'joy':(.8,.2),'love':(.8,.4),'nervousness':(-.4,.6),'optimism':(.6,.2),'pride':(.6,.1),'realization':(.2,.2),'relief':(.4,-.4),'remorse':(-.6,-.4),'sadness':(-.8,-.2),'surprise':(.2,.6),'neutral':(0,0)}
#moods=['energetic','excited','happy','relaxed','calm','depressed','sad','anxious']

In [27]:

def determineMoodLabel(valence,arousal):
    #determine the diagonal of the circumplex model that the valence and arousal scores fall on
    #MAKE 2 BOXES OF THE CIRCUMPLEX MODEL A MOOD 

    energetic =   -0.5<valence<0.5 and arousal>0.5
    happy =       valence>0.5 and -.5<arousal<0.5
    calm =       -0.5<valence<0.5 and arousal<-0.5
    sad =         valence<-0.5 and -.5<arousal<0.5

    excited =   not (happy or energetic) and valence>0 and arousal>0
    relaxed =   not (calm or happy) and valence>0 and arousal<0
    depressed = not (calm or sad) and valence<0 and arousal<0
    anxious =   not (energetic or sad) and valence<0 and arousal>0


    if energetic:
        mood='energetic'
    elif happy:
        mood='happy'
    elif calm:
        mood='calm'
    elif sad:
        mood='sad'
    elif excited:
        mood='excited'
    elif relaxed:
        mood='relaxed'
    elif depressed:
        mood='depressed'
    elif anxious:
        mood='anxious'
    else:
        mood='neutral'
    return mood     

def convertScoresToLabels(softmaxScores, emotionsAsValenceArousal,emotion_dict,printValenceArousal=False):
    #convert the softmax scores to a valence and arousal score
    #softmax scores are in the order of the values in emotion_dict so we can use emotion[id] to get the softmax value
    id=0
    valence=0
    arousal=0
    for i in softmaxScores:
        valence+=i*emotionsAsValenceArousal[emotion_dict[id]][0]
        arousal+=i*emotionsAsValenceArousal[emotion_dict[id]][1]
        id+=1
    #convert valence and arousal from tensor to float
    valence = valence.numpy()
    arousal = arousal.numpy()
    mood =determineMoodLabel(valence,arousal)
    if printValenceArousal:
        print("Valence: ",valence)
        print("Arousal: ",arousal)
    return mood

### Broken function but cleaner code (skip)

In [28]:
"""
def determineMoodLabel(valence, arousal):
    conditions = {
        'energetic': (-0.5 < valence < 0.5) and (arousal > 0.5),
        'happy': (valence > 0.5) and (-0.5 < arousal < 0.5),
        'calm': (-0.5 < valence < 0.5) and (arousal < -0.5),
        'sad': (valence < -0.5) and (-0.5 < arousal < 0.5),
    }
    conditions['excited']= (valence > 0) and (arousal > 0) and not ((conditions.get('happy') == True) or (conditions.get('energetic') == True))
    conditions['relaxed'] =(valence > 0) and (arousal < 0) and not ((conditions.get('calm')==True) or (conditions.get('happy') == True)),
    conditions['depressed']= (valence < 0) and (arousal < 0) and not ((conditions.get('calm')==True) or (conditions.get('sad') == True)),
    conditions['anxious']= (valence < 0) and (arousal > 0) and not ((conditions.get('energetic') == True) or (conditions.get('sad') == True))

    for mood, condition in conditions.items():
        #print(mood)
        if condition:
            if condition:
            #print(condition)
                return mood
    return 'neutral'
"""

"\ndef determineMoodLabel(valence, arousal):\n    conditions = {\n        'energetic': (-0.5 < valence < 0.5) and (arousal > 0.5),\n        'happy': (valence > 0.5) and (-0.5 < arousal < 0.5),\n        'calm': (-0.5 < valence < 0.5) and (arousal < -0.5),\n        'sad': (valence < -0.5) and (-0.5 < arousal < 0.5),\n    }\n    conditions['excited']= (valence > 0) and (arousal > 0) and not ((conditions.get('happy') == True) or (conditions.get('energetic') == True))\n    conditions['relaxed'] =(valence > 0) and (arousal < 0) and not ((conditions.get('calm')==True) or (conditions.get('happy') == True)),\n    conditions['depressed']= (valence < 0) and (arousal < 0) and not ((conditions.get('calm')==True) or (conditions.get('sad') == True)),\n    conditions['anxious']= (valence < 0) and (arousal > 0) and not ((conditions.get('energetic') == True) or (conditions.get('sad') == True))\n\n    for mood, condition in conditions.items():\n        #print(mood)\n        if condition:\n            if 

### Results

In [29]:
determineMoodLabel(1,1)

'excited'

In [30]:
convertScoresToLabels(softmaxtestUnclear, emotionsAsValenceArousal,emotion_dict,printValenceArousal=True)

Valence:  -0.00014550616
Arousal:  0.39976904


'anxious'

In [31]:
convertScoresToLabels(softmaxtestUnclear2, emotionsAsValenceArousal,emotion_dict,True)

Valence:  -6.28967e-06
Arousal:  4.1340345e-06


'anxious'

In [32]:
convertScoresToLabels(softmaxtestBraggingLyrics, emotionsAsValenceArousal,emotion_dict,True)

Valence:  0.17935042
Arousal:  0.0792139


'excited'

In [33]:
convertScoresToLabels(softmaxtestsadMetaphoricLyrics, emotionsAsValenceArousal,emotion_dict,True)

Valence:  -1.578913e-06
Arousal:  2.6779267e-06


'anxious'

In [34]:
convertScoresToLabels(softmaxtestgoEasyOnMeSad, emotionsAsValenceArousal,emotion_dict,True)

Valence:  -0.4687149
Arousal:  -0.37509394


'depressed'

# 4. Class Conversion Recap:

- The code works but it tends to classify heavily towards the center of the coordinates
- NOTE: This could be because it was just a few small samples, which may not reflect the overall distribution of the data
- As an alternative, we could remove the neutral score and apply softmax to the remaining classes
- OR, I could just exclude the inputs that result in overwhemingly neutral, and then continue to the next chunk of lyrics until I find all chunks that are not neutral and then get the average of those to get the final classification

# 5. Class Conversion With Special Neutral Considerations

First, in 6 and 7, we will try to get the average of the classes that are not overwhelmingly neutral to get final label.

If that doesnt work, here we will drop the neutral score and apply softmax to the remaining classes

- This would allow us to multiply the coordinates without shrinking the values due to the neutral score
- Perhaps this would allow us to reach the classes that are further away from the center
- If not, we could shift the boundaries of the classes closer to the center

# 6. Breaking up Lyrics to fit into model

In order to fit the lyrics into the model, we need to break up the lyrics. There are various ways to do this. We will try a few different methods and see which one works best.


- (Best) We could break it up by headers. Below are the various headers that we could use to break up the lyrics
https://genius.com/Genius-song-sections-and-headers-guide-annotated
    - in general, the intro is least emotional, the chorus is most emotional, and the verses are in between
    - This could mean we could skip the intro/outro and rely on the chorus and verses (maybe just the chorus when need fast results)

Some other ideas:
- We could break it up by sentences (some songs have no punctuation, so this may not work)
- We could break it up by line

### Song Samples used in testing (skip if not interested, but run cells)

Song sample 1.

This one is intentionally vague, but it is more relaxed or happy

In [35]:
fullLyricsLilTecca = """
[Intro]
Nah, nah, nah
Yeah, oh-oh
Yeah, oh no
We love you Tecca
Yeah, yeah
She just hit my phone, she said, "Tecca, you a winner" (That's Tec)
I just took her home, then I turn her to a sinner, yeah
Rambow

[Chorus]
She just hit my phone, she said, "Tecca, you a winner" (Yeah)
Took her home, then I turn her to a sinner, yeah
I just got a check, I ate lobster for dinner
She like my style, she tryna turn me to her nigga, yeah
I just woke up, now they say that I’m a star
I just wanna ball with my guys
Drive off the Wock', he just caught a DUI
He think he know the answers, nigga, like he Bill Nye
[Verse]
Bitch, I feel like A.I., nigga, never need to practice
I got two clips 'cause that chopper automatic
And I started talkin’ shit, I got tired of this rappin'
But I been making' hits, most these niggas, they been nappin'
And my lyrics hit her hard, like her nigga, he was stabbin'
LSD hit his brain, yeah, that nigga, he been tabbin'
I just got your shorty, yeah, my niggas, we been tappin'
Walk to the bank, Chief Keef, I be laughin'
Red, blue, yellow, nigga, you could pick your Power Ranger
I could fuck, but I won’t date ya
Getting bands, but I can’t save her
Fast nigga, but I can't chase her
360, quick scope, FaZe him
God-mode, aim-bot, laser
Cut you off, I got that razor

[Chorus]
She just hit my phone, she said, "Tecca, you a winner" (Yeah)
Took her home, then I turn her to a sinner, yeah
I just got a check, I ate lobster for dinner
She like my style, she tryna turn me to her nigga, yeah
I just woke up, now they sayin’ I'm a star
I just wanna ball with my guys
Drive off the Wock', he just caught a DUI
He think he got the answers, nigga, like he Bill Nye
"""

Song sample 2.

This one is more sad, and it is Go easy on me by Adele



In [36]:
fullLyricsAdele =""

### 6.0.1. Determining split when tokenizing lyrics exceeds 512 (model input limit)

The goal should be to split it as evenly as possible.

If over limit but under 1024, we can split it into 2 chunks, either at the nerest line break or paragraph break to the middle of the chunk

If it is bigger, we can divide by 512 to get n and then split it as smooth as possible into n+1 chunks

In [ ]:
tokens = tokenizer.encode(fullLyricsLilTecca)#, return_tensors="pt")
#tokens = tokens.cuda()
max_length = model.config.max_position_embeddings #aka 512

#if the token length over max length, get 

In [37]:
tokens = tokenizer.encode(fullLyricsLilTecca)#, return_tensors="pt")
#tokens = tokens.cuda()
max_length = model.config.max_position_embeddings #aka 512


if len(tokens) > max_length:
    songSegmentCount = int(len(tokens)/max_length)+1
    averageSegmentSize = int(len(tokens)/songSegmentCount)
    print(songSegmentCount)
    for i in range(1,songSegmentCount):
        #determine the smooth segment and break up lyrics by those indeces, returning it in a list
        print(averageSegmentSize)
        print(len(fullLyricsLilTecca[:averageSegmentSize]))

        
        #compare it for now to the len of tokens


        #need to think about how to go from encoded back to text. Maybe decode by the averageSegmentSize and then compare to original text to look for an end of line/new par!


    #Need to verify that when trying to jump to fit to a paragraph or sentence it doesn't accidentally go over the limit again!
    #possibly add a check to verify the new segments are not over the limit

    #after splitting it up, either have the funtion return the list of token segments or list of strings to be tokenized



print(songSegmentCount)


Token indices sequence length is longer than the specified maximum sequence length for this model (524 > 512). Running this sequence through the model will result in indexing errors


2
262
262
1678
2


In [44]:
# encode both fullLyricsAdele and fullLyricsLilTecca
tokens = tokenizer.encode([fullLyricsAdele,fullLyricsLilTecca], return_tensors="pt")

#keep track of the two songs


525

In [123]:
def breakUpLyricsToFitModel(fullLyricsString):
    songSegmentCount = 1
    songSegments=[fullLyricsString]
    tokenSegments = []

    #convert the string to a list of lines in the string
    lines = fullLyricsString.splitlines()[1:-1]#NOTE: THIS MIGHT NOT WORK WITH GENIUS LYRICS

    tokens = tokenizer.encode(fullLyricsString)
    max_length = model.config.max_position_embeddings #aka 512, could hard-code it to 512
    if len(tokens) > max_length:
        songSegments=[]
        songSegmentCount = int(len(tokens)/max_length)+1
        averageSegmentSize = int(len(tokens)/songSegmentCount)

        #for the number of songSegmentCount, decode the tokens by the average segment size to get the string
        for i in range(songSegmentCount):
            #decode the tokens by the average segment size to get the string
            startidx=averageSegmentSize*(i)
            if i == 0:
                startidx+=1
            endidx=averageSegmentSize*(i+1)
            if i == songSegmentCount-1:
                endidx-=1

            segment = tokenizer.decode(tokens[startidx:endidx])
            #if the segment is not at the end of a line, then find the next line break and add it to the segment
            if segment[-1] != ' ':


            songSegments.append(tokenizer.decode(tokens[startidx:endidx]))

    for segment in songSegments:

        #if cuda is available,  add to tokenSegments
        if torch.cuda.is_available():
            tokenSegments.append(tokenizer.encode(segment, return_tensors="pt").cuda())
        else:
            tokenSegments.append(tokenizer.encode(segment, return_tensors="pt"))

    #NOTE: I should add a check for the chunk size to make sure it is still not over the max length
    return tokenSegments, lines

In [122]:
x,lines = breakUpLyricsToFitModel(fullLyricsLilTecca)

[ Intro ] Nah, nah, nah Yeah, oh - oh Yeah, oh no We love you Tecca Yeah, yeah She just hit my phone, she said, " Tecca, you a winner " ( That's Tec ) I just took her home, then I turn her to a sinner, yeah Rambow [ Chorus ] She just hit my phone, she said, " Tecca, you a winner " ( Yeah ) Took her home, then I turn her to a sinner, yeah I just got a check, I ate lobster for dinner She like my style, she tryna turn me to her nigga, yeah I just woke up, now they say that I ’ m a star I just wanna ball with my guys Drive off the Wock ', he just caught a DUI He think he know the answers, nigga, like he Bill Nye [ Verse ] Bitch, I feel like A. I., nigga, never need to practice I got two clips'cause that chopper automatic And I started talkin ’ shit, I got tired of this rappin'But I been making'hits, most these niggas, they been nappin'And my lyrics hit her
-1
hard, like her nigga, he was stabbin'LSD hit his brain, yeah, that nigga, he been tabbin'I just got your shorty, yeah, my niggas, we

In [118]:
lines[0]

'[Intro]'

### 6.1 Breaking up by headers

Now that we accounted for the limit, we can break up the lyrics by headers and split large chunks if needed

# 7. Final Function For Inferences based on Lyrics

In [29]:
def getMoodLabelFromLyrics(model, lyrics, printRawScores=False, printTopN=False):
    #probably first need to pass through to the functions the various dictionaries like emotion_dict and emotionsAsValenceArousal


    #Deterime the lyrics breakdown (depending on how this is done, my for loop will either be chunks of tokens or lyrics)

    #for lyricsChunk in LyricsChunks:
        #convert the lyrics to tokens
        #feed tokens to model and Get the softmax
        #apply the softmax values to the valence and arousal values
        #get the average valence and arousal values
        #get the label of the appropriate range and keep track of it

    #return the most common label

IndentationError: expected an indented block (<ipython-input-29-7f41d4dde591>, line 14)